# Ethical Reminder

This notebook follows the **Lab Alliance Compact**. Proceed only if you commit to:

- Uphold data integrity: record results honestly and document processing steps.
- Credit original work: cite sources and acknowledge collaborators appropriately.
- Collaborate respectfully: communicate openly, share workload fairly, and seek help early when challenges arise.

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

In [ ]:
rng = np.random.default_rng(42)
x = np.linspace(0, 10, 25)
true_m, true_b = 2.5, -1.0
stat_sigma = 0.6
systematic_offset = 0.3
noise = rng.normal(0, stat_sigma, size=x.size)
y = true_m * x + true_b + noise + systematic_offset

stat_uncertainty = np.full_like(x, stat_sigma)

In [ ]:
def linear(x, m, b):
    return m * x + b

popt, pcov = curve_fit(linear, x, y, sigma=stat_uncertainty, absolute_sigma=True)
perr = np.sqrt(np.diag(pcov))
print(f"Slope: {popt[0]:.3f} ± {perr[0]:.3f}")
print(f"Intercept: {popt[1]:.3f} ± {perr[1]:.3f}")

In [ ]:
fit_y = linear(x, *popt)
residuals = y - fit_y

fig, axes = plt.subplots(2, 1, figsize=(6, 8), sharex=True)
axes[0].errorbar(x, y, yerr=stat_uncertainty, fmt='o', label='Measurements')
axes[0].plot(x, fit_y, '-', label='Best-fit line')
axes[0].set_ylabel('y')
axes[0].legend()

axes[1].axhline(0, color='k', linestyle='--')
axes[1].errorbar(x, residuals, yerr=stat_uncertainty, fmt='o')
axes[1].set_xlabel('x')
axes[1].set_ylabel('Residuals')
plt.tight_layout()
plt.show()

## Discussion

The statistical uncertainties (error bars) describe measurement scatter and inform the weighted fit.
The systematic offset of 0.3 introduces a bias in the intercept, visible in the residuals trending positive.
Further investigation could include independent calibration or control measurements to constrain systematic effects.